# lasagne
* lasagne is a library for neural network building and training
* it's a low-level library with almost seamless integration with theano

For a demo we shall solve the same digit recognition problem, but at a different scale
* images are now 28x28
* 10 different digits
* 50k samples

In [11]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print(X_train.shape,y_train.shape)

(50000, 1, 28, 28) (50000,)


In [13]:
input_X = T.tensor4("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = (None,1,28,28)

target_y = T.vector("target Y integer",dtype='int32')

Defining network architecture

In [14]:
import lasagne
from lasagne.layers import *

#Input layer (auxilary)
l1 = InputLayer(shape = input_shape, input_var=input_X)

l2 = DropoutLayer(l1, p = 0.2)

l3 = Conv2DLayer(l2, num_filters=20, stride = 1, filter_size=(5,5) , nonlinearity=lasagne.nonlinearities.leaky_rectify)

lmaxpull = MaxPool2DLayer(l3, pool_size=(2,2))

l4 = DenseLayer(lmaxpull,num_units=350 ,nonlinearity = lasagne.nonlinearities.leaky_rectify)

l_out = DenseLayer(dropout(l4, 0.5),num_units = 10,nonlinearity=lasagne.nonlinearities.softmax)

In [15]:
#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(l_out, deterministic = False)
y_predicted_dropout_off = lasagne.layers.get_output(l_out, deterministic = True)

In [16]:
#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(l_out)
print (all_weights)

[W, b, W, b, W, b]


### Than you could simply
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a whole lot of work and life's short
  * not to mention life's too short to wait for SGD to converge

Instead, we shall use Lasagne builtins

In [17]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
accuracy_test = lasagne.objectives.categorical_accuracy(y_predicted_dropout_off,target_y).mean()
#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.sgd(loss, all_weights,learning_rate=0.03)

In [18]:
#function that computes loss and updates weights
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#function that just computes accuracy
accuracy_fun = theano.function([input_X,target_y],accuracy)
accuracy_fun_test = theano.function([input_X,target_y],accuracy_test)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [19]:
# An auxilary function that returns mini-batches for neural network training

# Parameters
# inputs - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# outputs - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches

def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

# Training loop

In [20]:
import time

num_epochs = 100 #amount of passes through the data

batch_size = 50 #number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    i = 0
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    if (epoch+1)%5 == 0:
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, batch_size):
            inputs, targets = batch
            val_acc += accuracy_fun(inputs, targets)
            val_batches += 1

    
        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))

        print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
        print("  train accuracy:\t\t{:.2f} %".format(
            train_acc / train_batches * 100))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

Epoch 5 of 100 took 157.031s
  training loss (in-iteration):		0.137951
  train accuracy:		95.76 %
  validation accuracy:		96.31 %
Epoch 10 of 100 took 157.745s
  training loss (in-iteration):		0.087707
  train accuracy:		97.32 %
  validation accuracy:		97.06 %
Epoch 15 of 100 took 200.301s
  training loss (in-iteration):		0.064212
  train accuracy:		98.08 %
  validation accuracy:		97.67 %
Epoch 20 of 100 took 249.241s
  training loss (in-iteration):		0.052715
  train accuracy:		98.30 %
  validation accuracy:		97.93 %
Epoch 25 of 100 took 156.143s
  training loss (in-iteration):		0.044547
  train accuracy:		98.55 %
  validation accuracy:		97.94 %
Epoch 30 of 100 took 258.485s
  training loss (in-iteration):		0.037011
  train accuracy:		98.83 %
  validation accuracy:		98.09 %
Epoch 35 of 100 took 291.475s
  training loss (in-iteration):		0.032007
  train accuracy:		98.95 %
  validation accuracy:		98.20 %
Epoch 40 of 100 took 263.007s
  training loss (in-iteration):		0.028709
  train accu

In [21]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun_test(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		99.13 %
Achievement unlocked: 80lvl Warlock!


# Now improve it!

* Moar layers!
* Moar units!
* Different nonlinearities!